# Setup Environment in Google Colab
Configure Google Colab environment, check for GPU availability, and mount Google Drive for storage.

In [2]:
# Check if GPU is available
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('GPU is not available, using CPU')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to the project folder in Google Drive
import os
project_path = '/content/drive/MyDrive/3R1CSegmentation'
if not os.path.exists(project_path):
    os.makedirs(project_path)
os.chdir(project_path)
print(f'Changed working directory to {project_path}')

GPU is available
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed working directory to /content/drive/MyDrive/3R1CSegmentation


# Install YOLOv8
Install the Ultralytics YOLOv8 package and other required dependencies.

In [3]:
# Install YOLOv8
!pip install ultralytics

# Verify the installation
!yolo checks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

# Prepare Clothing Dataset
Download deepFashion 2 dataset from https://github.com/switchablenorms/DeepFashion2.
I have uploaded a subset ok the dataset which consist of 10k for training and 4386 images for validation (70% training, 30% validation)

Also I have uploaded a utility tool (deepfashion2_to_yolo.py) to my github for coverting the format to yolo segmentation format.


Turn deepfasion2 data to yolo format
convert_deepfashion2_to_yolo(input_dir,images_dir, output_dir)

In [ ]:
#for testing purpose
!python deepfashion2_to_yolo.py datasets/coco/train_subset/annos datasets/coco/train_subset/images datasets/yolo/train/labels
!python deepfashion2_to_yolo.py datasets/coco/val_subset/annos datasets/coco/val_subset/images datasets/yolo/val/labels

Conversion completed. YOLO format annotations saved to datasets/yolo/train/labels
Conversion completed. YOLO format annotations saved to datasets/yolo/val/labels


In [ ]:
%%bash

# Define source and destination directories
source_dir="train/train/image"
destination_dir="datasets/train_subset/images"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.jpg"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 images from $source_dir to $destination_dir."

Copied the first 10,000 images from train/train/image to datasets/train_subset/images.


In [ ]:
%%bash

# Define source and destination directories
source_dir="train/train/annos"
destination_dir="datasets/train_subset/annos"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.json"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 annos from $source_dir to $destination_dir."

Copied the first 10,000 annos from train/train/annos to datasets/train_subset/annos.


In [ ]:
%%bash

# Define source and destination directories
source_dir="validation/validation/image"
destination_dir="datasets/val_subset/images"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.jpg"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 images from $source_dir to $destination_dir."

Copied the first 10,000 images from validation/validation/image to datasets/val_subset/images.


In [ ]:
%%bash

# Define source and destination directories
source_dir="validation/validation/annos"
destination_dir="datasets/val_subset/annos"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.json"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 annos from $source_dir to $destination_dir."

Copied the first 10,000 annos from validation/validation/annos to datasets/val_subset/annos.


In [ ]:
#convert the subset to yolo format
!python deepfashion2_to_yolo.py datasets/train_subset/annos datasets/train_subset/images datasets/train_subset/labels
!python deepfashion2_to_yolo.py datasets/val_subset/annos datasets/val_subset/images datasets/val_subset/labels

Conversion completed. YOLO format annotations saved to datasets/train_subset/labels
Conversion completed. YOLO format annotations saved to datasets/val_subset/labels


# Train the Segmentation Model
Train YOLOv8 segmentation model to identify clothing items and create masks for background removal.

In [13]:
# Train the Segmentation Model

# Import necessary libraries
import torch
from ultralytics import YOLO

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('GPU is not available, using CPU')


# Initialize the YOLOv8 model for segmentation
model_segmentation = YOLO('yolo11s-seg.pt')  # Use a pre-trained YOLOv11 model for segmentation
# model_segmentation.tune(data="yoloTune.yaml", epochs=50, iterations=300, optimizer="AdamW", imgsz=640 ,batch=0.7)

# Configure the model for training
model_segmentation.train(data='yoloSeg.yaml', epochs=100, imgsz=640,batch=0.7,optimizer="AdamW")

print('YOLOv11 segmentation model training completed.')

GPU is available
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=yoloSeg.yaml, epochs=100, time=None, patience=100, batch=0.7, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train123, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/labels.cache... 10000 images, 0 backgrounds, 56 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000013.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000014.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000115.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000117.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000122.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000129.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011]
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation

      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    10087319       35.62         2.496         40.76           nan        (1, 3, 640, 640)                    list
    10087319       71.24         2.682         38.21           nan        (2, 3, 640, 640)                    list
    10087319       142.5         4.333         42.18           nan        (4, 3, 640, 640)                    list
    10087319       284.9         6.967         45.63           nan        (8, 3, 640, 640)                    list
    10087319       569.9        12.558         50.17           nan       (16, 3, 640, 640)                    list
    10087319        1140        22.993         56.04           nan       (32, 3, 640, 640)                    list
    10087319        2280        44.218         95.24           nan       (64, 3, 640, 640)                    list
AutoBatch: Using batch-size 35 for CUDA:0 30.62G/39.56G (77%) ✅


train: Scanning /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/labels.cache... 10000 images, 0 backgrounds, 56 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   3%|▎         | 2/62 [00:01<00:52,  1.15it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:29<00:00,  2.12it/s]


                   all       4286       6776      0.526      0.381       0.37      0.303      0.459       0.27      0.261      0.162
   short_sleeved_shirt       1340       1405      0.321      0.754      0.647      0.581      0.317      0.742      0.636      0.356
    long_sleeved_shirt        796        919      0.328      0.392      0.354      0.291      0.301      0.358      0.305     0.0904
 short_sleeved_outwear          6          8          1          0    0.00573     0.0038          1          0   0.000768    0.00028
  long_sleeved_outwear        232        396      0.357      0.351      0.259      0.179      0.153      0.149     0.0726      0.016
                  vest        258        310      0.467      0.439      0.426      0.365      0.468      0.439      0.422       0.29
                 sling         63        114          1          0     0.0626     0.0433          1          0     0.0569     0.0374
                shorts        290        300      0.323      0.643   

In [ ]:
model_segmentation.tune(data="yoloTune.yaml", epochs=50, iterations=300, optimizer="AdamW", imgsz=640 ,batch=0.7,resume=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-6d5bb5b4fd68>", line 1, in <cell line: 0>
    model_segmentation.tune(data="yoloTune.yaml", epochs=50, iterations=300, optimizer="AdamW", imgsz=640 ,batch=0.7)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 846, in tune
    return Tuner(args=args, _callbacks=self.callbacks)(model=self, iterations=iterations)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/tuner.py", line 97, in __init__
    self.tune_dir = get_save_dir(self.args, name=self.args.name or "tune")
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/cfg/__init__.py", line 416, in get_save_dir
    save_dir = increment_

In [ ]:
model_segmentation.train(data='yoloSeg.yaml', epochs=100, imgsz=640,batch=0.7,optimizer="AdamW",
    lr0=0.00726,
    lrf=0.01186,
    momentum=0.91123,
    weight_decay=0.00034,
    warmup_epochs=2.47414,
    warmup_momentum=0.77204,
    box=7.49251,
    cls=0.52952,
    dfl=1.40377,
    hsv_h=0.01952,
    hsv_s=0.62972,
    hsv_v=0.32967,
    degrees=0.0,
    translate=0.0997,
    scale=0.34433,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.32907,
    bgr=0.0,
    mosaic=0.94664,
    mixup=0.0,
    copy_paste=0.0)

Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=yoloSeg.yaml, epochs=100, time=None, patience=100, batch=0.7, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train1232, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/labels.cache... 10000 images, 0 backgrounds, 56 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000013.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000014.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000115.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000117.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000122.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/images/000129.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011]
train: WARNING ⚠️ /content/drive/MyDrive/3R1CSegmentation

      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    10087319       35.62         2.483         34.95           nan        (1, 3, 640, 640)                    list
    10087319       71.24         2.718         43.07           nan        (2, 3, 640, 640)                    list
    10087319       142.5         4.349         39.15           nan        (4, 3, 640, 640)                    list
    10087319       284.9         7.011         47.92           nan        (8, 3, 640, 640)                    list
    10087319       569.9        12.667         48.63           nan       (16, 3, 640, 640)                    list
    10087319        1140        22.922         57.68           nan       (32, 3, 640, 640)                    list
    10087319        2280        44.128         95.03           nan       (64, 3, 640, 640)                    list
AutoBatch: Using batch-size 48 for CUDA:0 28.15G/39.56G (71%) ✅


train: Scanning /content/drive/MyDrive/3R1CSegmentation/datasets/train_subset/labels.cache... 10000 images, 0 backgrounds, 56 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]

...

                   all       4286       6776      0.363      0.423      0.359      0.296      0.029     0.0428    0.00707    0.00275

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     42/100      18.5G     0.7739      1.853     0.8183     0.9817         54        640: 100%|██████████| 208/208 [01:22<00:00,  2.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:27<00:00,  1.62it/s]


                   all       4286       6776      0.425      0.381      0.358      0.295     0.0232     0.0336    0.00476    0.00182

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     43/100      18.5G     0.7699      1.852     0.8222     0.9808        370        640:  86%|████████▌ | 178/208 [01:09<00:11,  2.56it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-449b339b38d6>", line 1, in <cell line: 0>
    model_segmentation.train(data='yoloSeg.yaml', epochs=100, imgsz=640,batch=0.7,optimizer="AdamW",
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 791, in train
    self.trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 211, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 367, in _do_train
    for i, batch in pbar:
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/data/build.py", line 59, in __iter__
    yield next(self.iterator)
          ^^^^^^^^

# Evaluate Model Performance
Check the generated result.png to evaluate the trained model.

# Perform Background Removal
Implement a pipeline to use the trained segmentation model to remove backgrounds from clothing images.

In [ ]:
# Perform Background Removal

# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Function to remove background using the segmentation model
def remove_background(image_path, model):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Preprocess the image for the model
    input_image = cv2.resize(image_rgb, (640, 640))
    input_image = input_image / 255.0
    input_image = np.transpose(input_image, (2, 0, 1))
    input_image = np.expand_dims(input_image, axis=0)
    input_image = torch.tensor(input_image, dtype=torch.float32).to(device)

    # Get the segmentation mask from the model
    with torch.no_grad():
        output = model(input_image)[0]
    mask = torch.argmax(output, dim=1).cpu().numpy()[0]

    # Resize the mask to the original image size
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Create a binary mask
    binary_mask = (mask > 0).astype(np.uint8)

    # Apply the mask to the image
    result = cv2.bitwise_and(image_rgb, image_rgb, mask=binary_mask)

    return result

# Example usage
image_path = 'path_to_your_image.jpg'  # Replace with the path to your image
result = remove_background(image_path, model_segmentation)

# Display the original image and the result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Background Removed')
plt.imshow(result)
plt.axis('off')

plt.show()

# Export and Save the Models
Export the trained models in appropriate formats (ONNX, TorchScript, etc.) and save them to Google Drive for future use.

In [ ]:
# Export and Save the Models

# Export the trained classification model to ONNX format
classification_onnx_path = os.path.join(project_path, 'yolov8_classification.onnx')
model_classification.export(format='onnx', path=classification_onnx_path)
print(f'Classification model exported to {classification_onnx_path}')

# Export the trained segmentation model to ONNX format
segmentation_onnx_path = os.path.join(project_path, 'yolov8_segmentation.onnx')
model_segmentation.export(format='onnx', path=segmentation_onnx_path)
print(f'Segmentation model exported to {segmentation_onnx_path}')

# Save the trained classification model to TorchScript format
classification_torchscript_path = os.path.join(project_path, 'yolov8_classification.pt')
model_classification.save(path=classification_torchscript_path)
print(f'Classification model saved to {classification_torchscript_path}')

# Save the trained segmentation model to TorchScript format
segmentation_torchscript_path = os.path.join(project_path, 'yolov8_segmentation.pt')
model_segmentation.save(path=segmentation_torchscript_path)
print(f'Segmentation model saved to {segmentation_torchscript_path}')

# Save models to Google Drive
drive_classification_onnx_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_classification.onnx'
drive_segmentation_onnx_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_segmentation.onnx'
drive_classification_torchscript_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_classification.pt'
drive_segmentation_torchscript_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_segmentation.pt'

shutil.copy(classification_onnx_path, drive_classification_onnx_path)
shutil.copy(segmentation_onnx_path, drive_segmentation_onnx_path)
shutil.copy(classification_torchscript_path, drive_classification_torchscript_path)
shutil.copy(segmentation_torchscript_path, drive_segmentation_torchscript_path)

print('Models saved to Google Drive.')